In [50]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_aer.noise import NoiseModel, pauli_error
import numpy as np

def qft(qc, n):
    for j in range(n):
        qc.h(j)
        for k in range(j + 1, n):
            qc.cp(np.pi / (2 ** (k - j)), k, j)

def quantum_sum(a, b):
    n = max(len(bin(a)) - 2, len(bin(b)) - 2)
    qc = QuantumCircuit(n * 2, n * 2)  # Allocate classical bits for measurement
    for i in range(len(bin(a)) - 2):
        if (a >> i) & 1:
            qc.x(i)
    for i in range(len(bin(b)) - 2):
        if (b >> i) & 1:
            qc.x(i + n)
    qft(qc, n)
    qc.measure(range(n * 2), range(n * 2))  # Add measurement
    return qc

def add_noise(p1, p2, circuit):
    noise_model = NoiseModel()
    pauli_1q_error = pauli_error([('X', p1), ('Y', p1), ('Z', p1), ('I', 1 - 3 * p1)])
    noise_model.add_all_qubit_quantum_error(pauli_1q_error, ['u1', 'u2', 'u3', 'rz', 'sx', 'x'])
    pauli_2q_error = pauli_error([('II', 1 - 3 * p2), ('XX', p2), ('YY', p2), ('ZZ', p2)])
    noise_model.add_all_qubit_quantum_error(pauli_2q_error, ['cx'])
    return circuit, noise_model

def analyze_noise_effects(a, b, p1, p2):
    qc = quantum_sum(a, b)
    qc_transformed = transpile(qc, basis_gates=['u1', 'u2', 'u3', 'rz', 'sx', 'x', 'cx'])
    noisy_circuit, noise_model = add_noise(p1, p2, qc_transformed)
    backend = Aer.get_backend('qasm_simulator')
    job = backend.run(noisy_circuit, noise_model=noise_model, shots=1024)
    counts = job.result().get_counts()
    return counts

a = 1
b = 2
p1 = 0.01
p2 = 0.01
counts = analyze_noise_effects(a, b, p1, p2)
print(counts)


{'0010': 6, '1011': 266, '0001': 1, '1001': 255, '1010': 252, '0000': 4, '1000': 235, '0011': 5}
